<a href="https://colab.research.google.com/github/Jderrick97/Colab/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install talos
import talos
from talos import Scan, Evaluate, Deploy


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend
from tensorflow.keras.activations import softmax, sigmoid, relu
from tensorflow.keras.optimizers import Adam


import pickle




#Gathering the data as described by the instructions

with open('train', 'rb') as file:
    train_dict = pickle.load(file, encoding='bytes')

with open('test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

X_train = train_dict[b'data']
y_train = train_dict[b'coarse_labels']

X_test = test_dict[b'data']
y_test = test_dict[b'coarse_labels']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

p = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'activation': ['softmax', 'sigmoid'],
    'loss': ['mse', 'categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [1000, 2000]
}




#The Sequential model with the added hyperparameters to be scanned

def my_model(X_train,y_train,X_test,y_test,params):
    model = Sequential()
    model.add(Dense(units = params['units'], input_shape = X_train.shape[1:]))
    model.add(Dense(units = params['units'], activation = params['hidden_activations']))
    model.add(Dense(units = params['units'], activation = params['hidden_activations']))
    model.add(Dense(units = params['units'], activation = params['hidden_activations']))
    model.add(Dense(units = params['units'], activation = params['hidden_activations']))
    model.add(Dense(units = params['units'], activation = params['hidden_activations']))
    model.add(Dense(units = 100, activation = params['activation']))
    
    model.compile(loss=params['loss'], 
                  optimizer = params['optimizer'], 
                  metrics = ['accuracy'])
    
    out = model.fit(x=X_train, 
                    y=y_train, 
                    batch_size=params['batch_size'], 
                    epochs=200)
    
    return out, model



#Implementation of Talos to scan through hyperparameters to determine the best

tal_scan = talos.Scan(x = X_train, 
               y = y_train, 
               params = p, 
               model = my_model, 
               x_val = X_test, 
               y_val = y_test, 
               experiment_name = "MLP Hyperparameter Table")

evalu = talos.Evaluate(tal_scan)

evaluation = evalu.evaluate(X_test, y_test, metric = 'accuracy', task = 'multi_label')

print("The Evaluations Are:")

print(evaluation[0])
print(evaluation[1])
print(evaluation[2])
print(evaluation[3])
print(evaluation[4])

model = Deploy(tal_scan, model_name = 'CIFAR-100 MLP', metric = 'accuracy')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1/200
50/50 [==============================] - 5s 59ms/step - loss: 0.0095 - accuracy: 0.0940
Epoch 2/200
50/50 [==============================] - 4s 88ms/step - loss: 0.0089 - accuracy: 0.1865
Epoch 3/200
28/50 [===============>..............] - ETA: 2s - loss: 0.0087 - accuracy: 0.2206

KeyboardInterrupt: ignored